In [39]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd


In [40]:
con = pymysql.connect(host='localhost', user='root', password='1234',
                      db='MySQl', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True, # 결과 DB 반영 (Insert or update)
                      cursorclass=pymysql.cursors.DictCursor # DB조회시 컬럼명을 동시에 보여줌
                     )
con.cursor()

In [38]:
# cur = con.cursor()
# # sql = "SELECT * FROM stock_info.stock" # customers 테이블 전체를 불러옴
# sql = "SELECT * FROM stock_info.stock.columns" # customers 테이블 전체를 불러옴
# cur.execute(sql)
# rows = cur.fetchone()
# con.close() # DB 연결 종료
# print(rows)

In [43]:
test = pd.read_csv('../data/000250_삼천당제약.csv')
test.to_sql(name='trad',con=con, if_exists='append', index=False)

c:\Users\TJ\anaconda3\envs\trad\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': not all arguments converted during string formatting